In [54]:
import pandas as pd
import os
import re

In [53]:
weeks_dir = '../../Clean_Data/Weeks'

In [65]:
def extract_yardage(play_description):
    match = re.search(r'FOR (-?\d+) YARDS?', play_description)
    if match:
        return int(match.group(1))
    else:
        return 0

In [66]:
# Sets play_type & IsRush accordingly, using yards function to extract and set yards gained
def fix_play_row(play):
    play['PlayType'] = 'RUSH'
    play['IsRush'] = 1
    yards = extract_yardage(play['Description'])
    play['Yards'] = yards
    
    return play

In [86]:
def find_direct_snap(df, direct_snaps):
    for index, play in df.iterrows():
        if pd.isna(play['PlayType']) and not (
        play['Description'].startswith("TIMEOUT") or
        play['Description'] == 'TWO-MINUTE WARNING' or
        play['Description'].startswith('END')
        ):
            play_fixed = fix_play_row(play)
            direct_snaps.append(play_fixed)
            df.loc[index] = play_fixed


In [93]:
direct_snaps = []

for week in os.listdir(weeks_dir):
    week_dir = os.path.join(weeks_dir, week)

    for file in os.listdir(week_dir):
        week_df = pd.read_csv(os.path.join(week_dir, file))
        find_direct_snap(week_df, direct_snaps)    
 

In [94]:
print(len(direct_snaps))

72


In [70]:
direct_snap_df = pd.DataFrame(direct_snaps)

direct_snap_df.to_csv('direct_snaps.csv', index=False)
